<a href="https://colab.research.google.com/github/lakshitgosain/Tensorflow-ZTM/blob/main/TF_ZTM_04_Transfer_learning(Feature_Extraction).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Can Leverage an existing Architecture proven to provide good results on different Datasets
* We can leverage a working network architecture which has already learned patterns on similar data to our own